In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
def compute_adjacency_matrix(df):
    # Xác định số lượng người dùng và số lượng phim duy nhất từ dữ liệu
    unique_users = df.user_id.unique()
    unique_movies = df.movie_id.unique()
    num_users = len(unique_users)
    num_movies = len(unique_movies)

    # Tạo ma trận kề ban đầu với kích thước tương ứng với số lượng người dùng và phim
    adjacency_matrix = np.zeros((num_users, num_movies))

    # Đánh dấu 1 cho các liên kết giữa người dùng và phim trong dữ liệu Netflix
    for user, movie in df[['user_id', 'movie_id']].values:
        user_index = np.where(unique_users == int(user))[0][0]
        movie_index = np.where(unique_movies == int(movie))[0][0]
        adjacency_matrix[user_index, movie_index] = 1

    return np.array(adjacency_matrix)

In [3]:
def compute_user_weight_matrix(df):
    # Xác định số lượng người dùng và số lượng phim duy nhất từ dữ liệu
    unique_users = df.user_id.unique()
    unique_movies = df.movie_id.unique()
    num_users = len(unique_users)
    num_movies = len(unique_movies)

    # Tạo ma trận trọng số giữa người dùng với các người dùng khác
    weight_matrix = np.zeros((num_users, num_users))
    source_target = []

    # Tính toán trọng số giữa người dùng với các người dùng khác
    for i in range(num_users):
        for j in range(i+1, num_users):
            user1 = unique_users[i]
            user2 = unique_users[j]
            
            # Tính số lượng phim chung đã xem
            common_movies = set()
            common_movies_user_1 = set()
            common_movies_user_2 = set()
            for user, movie in df[['user_id', 'movie_id']].values:
                if user == user1:
                    common_movies_user_1.add(int(movie))
                elif user == user2:
                    common_movies_user_2.add(int(movie))
                common_movies = common_movies_user_1.intersection(common_movies_user_2)

            # Gán trọng số tương tự nhau dựa trên số lượng phim chung đã xem
            weight_matrix[i, j] = len(common_movies)
            weight_matrix[j, i] = len(common_movies)
            source_target.append([user1, user2, weight_matrix[i][j]])

    return weight_matrix, np.array(source_target)

In [4]:
def compute_sum_tot(source_target, source, target, update_source_target = None):
    sum_source = 0
    sum_target = 0
    value_source_target = 0
    if update_source_target == None: 
        for item in source_target:
            if item[0] == source:
                sum_source = sum_source + item[2]
            elif item[0] == target:
                sum_target = sum_target + item[2]
            if item[0] == source and item[1] == target:
                value_source_target = value_source_target + item[2]
    else:
        source= np.array(source)
        target= np.array(target)
        for key, item in update_source_target.items():
            if item[0] in source:
                sum_source = sum_source + item[2]
            elif item[0] in target:
                sum_target = sum_target + item[2]
            if item[0] in source and item[1] in target:
                value_source_target = value_source_target + item[2]
    return sum_source + sum_target - value_source_target

In [5]:
def compute_k_i(source_target, source, target, update_source_target = None):
    k_i = 0

    if update_source_target == None:
        for item in source_target:
            if item[0] == source and item[1] != target:
                k_i = k_i + 1
            elif item[0] == target:
                k_i = k_i + 1
            elif item[1] == target and item[0] != source:
                k_i = k_i + 1
            elif item[1] == source:
                k_i = k_i + 1
    else:
        source= np.array(source)
        target= np.array(target)
        for key, item in update_source_target.items():
            # print(f'user1: {item[0]} --------- user2: {item[1]}')
            if ((len(item[1]) > 1 or len(item[0]) > 1) and (item[0] in source or item[1] in source or item[0] in target or item[1] in target)) :
                if (item[0] in source and item[1] in target):
                    continue
                else: 
                    if len(item[0]) > 1:
                        k_i = k_i + len(item[0]) - 1  
                    if len(item[1]) > 1:
                        k_i = k_i + len(item[1]) - 1   
                    # print(k_i)
            if item[0] in source and item[1] not in target:
                k_i  = k_i  + 1 ## 4
                # print(k_i)
            elif item[0] in target:
                k_i = k_i + 1 ## 4
                # print(k_i)
            elif item[1] in target and item[0] not in source:
                k_i = k_i + 1
                # print(k_i)
            elif item[1] in source:
                k_i = k_i + 1
                # print(k_i)
    return k_i

In [6]:
def sum_in_value(source_target, target):
    sum_in_value = 0
    if len(target) > 1: 
        for i in range(len(target)):
            for j in range(i + 1, len(target)):
                sum_in_value = sum_in_value + source_target[np.where((source_target[:, 0] == target[i]) & (source_target[:, 1] == target[j]))[0][0]][2]
    return sum_in_value

In [7]:
def k_in_value(group):
    k_in_value = 0
    if len(group) > 1: 
            for i in range(len(group)):
                for j in range(i + 1, len(group)):
                    # print(f"{group[i]}-------{group[j]}")
                    k_in_value = k_in_value + 1
    return k_in_value

In [8]:
def compute_k_i_in(source_target, source, group):
    if len(group) <= 1:
        k_i_in = 1
    else: 
        k_i_in = 0
        source = np.array(source)
        group = np.array(group)
        for item in source_target:
            # print(f"user1: {item[0]} --------- user2: {item[1]}")
            if (item[0] in source and item[1] in group) :
                k_i_in = k_i_in + 1
                # print(k_i_in)
    return k_i_in + k_in_value(group)

In [9]:
def compute_2_m(source_target, weight_matrix, update_source_target = None):
    if update_source_target == None: 
        return np.sum(weight_matrix)
    else:
        _2m = 0
        for key, item in update_source_target.items():
            _2m = _2m + item[2]
        return 2 * _2m

In [10]:
def check_in_append(value_node, array_node):
    count = 0
    for i in value_node:
        for j in range(len(array_node)):
            if i in array_node[j]:
                count = count + 1
                break
    if count == 0:
        array_node.append(value_node)

In [11]:
def check_in(value_node, usr):
    count = 0
    for i in value_node:
        if i in usr:
            count = count + 1
            break
    if count == 0:
        return 0 #chua co
    else:
        return 1 #co

In [12]:
def the_number_of_node_in_steps(df, partition, choose_node = None):
    unique_users = df.user_id.unique()
    if len(partition) == 0:
            # unique_users = df.user_id.unique()
            usr = []
            for i in unique_users:
                usr.append([i])
            partition[0] = usr
    else:
            if len(choose_node) != 0:
                array_node = []
                for key, value in choose_node.items():
                    if len(array_node) == 0:
                        array_node.append(value[0])
                    else:
                        check_in_append(value[0], array_node)
                # print(array_node)
            usr = []
            if len(partition) == 1:
                for i in range(len(array_node)):
                    for j in range(len(array_node[i])):
                        usr.append(array_node[i][j])
                usr1 = [usr]
                for i in unique_users:
                    if check_in([i], usr) == 0:
                        usr1.append([i])
                partition[len(partition)] = usr1
            else:
                for i in range(len(array_node)):
                    for j in range(len(array_node[i])):
                        for k in range(len(array_node[i][j])):
                            usr.append(array_node[i][j][k])
                usr1 = [usr]
                for i in unique_users:
                    # print(i)
                    if check_in([i], usr) == 0:
                        usr1.append([i])
                partition[len(partition)] = usr1

In [13]:
def update_weight_source_target(partition, source_target):
    # Xác định số lượng người dùng và số lượng phim duy nhất từ dữ liệu
    last_key = list(partition)[-1]
    unique_users = partition[last_key]
    unique_users.sort()
    num_users = len(unique_users)


    # Tạo ma trận trọng số giữa người dùng với các người dùng khác
    update_source_target = {}
         
    # Tính toán trọng số giữa người dùng với các người dùng khác
    count = 0
    for i in range(num_users):
      for j in range(i+1, num_users):
            user1 = unique_users[i]
            user2 = unique_users[j]

            # print(f'user1: {user1} ----------------- user2: {user2}')
            weight_value = 0
            for item1 in user1:
                  for item2 in user2:
                        weight_value = weight_value + source_target[np.where((source_target[:, 0] == item1) & (source_target[:, 1] == item2))[0][0]][2]
            update_source_target[count] = [user1, user2, weight_value]
            count = count + 1

    return update_source_target

In [14]:
def louvain_algorithm(df, weight_matrix, source_target, partition, update_source_target = None):
    while True:
        dict_partition = {}
        if update_source_target == None:
            for i in range(len(source_target)):
                user_group = [source_target[i][0], source_target[i][1]]
                sum_in = source_target[i][2] + sum_in_value(source_target, [source_target[i][1]])
                sum_tot = compute_sum_tot(source_target, source_target[i][0], source_target[i][1])
                k_i_in = compute_k_i_in(source_target, [source_target[i][0]], [source_target[i][1]]) 
                k_i = compute_k_i(source_target, source_target[i][0], source_target[i][1])
                _2m = compute_2_m(source_target, weight_matrix)
                Q = ((sum_in + k_i_in)/(_2m) - ((sum_tot + k_i)/_2m)**2 - (sum_in/_2m - ((sum_tot/_2m)**2) - ((k_i)/_2m)**2))
                dict_partition[i] = [user_group, sum_in, sum_tot, k_i_in, k_i, _2m, Q]
        else:
            for i in range(len(update_source_target)):
                user_group = [update_source_target[i][0], update_source_target[i][1]]
                sum_in = update_source_target[i][2] + sum_in_value(source_target, update_source_target[i][1])
                sum_tot = compute_sum_tot(source_target, update_source_target[i][0], update_source_target[i][1], update_source_target)
                k_i_in = compute_k_i_in(source_target, update_source_target[i][0], update_source_target[i][1])
                k_i = compute_k_i(source_target, update_source_target[i][0], update_source_target[i][1], update_source_target)
                _2m = compute_2_m(source_target, weight_matrix, update_source_target)
                Q = ((sum_in + k_i_in)/(_2m) - ((sum_tot + k_i)/_2m)**2 - (sum_in/_2m - ((sum_tot/_2m)**2) - ((k_i)/_2m)**2))
                dict_partition[i] = [user_group, sum_in, sum_tot, k_i_in, k_i, _2m, Q]
        positive_values = {key: value for key, value in dict_partition.items() if value[-1] > 0}
        sorted_data = sorted(positive_values.items(), key=lambda x: x[-1], reverse=True)
        choose_node = {}
        for key, value in sorted_data:
            if value[-1] > 0:
                choose_node[key] = value
        the_number_of_node_in_steps(df, partition, choose_node)
        update_source_target = update_weight_source_target(partition, source_target)
        if len(update_source_target) == 0:
            break

In [15]:
df = pd.read_csv("C:/Users/UYEN/OneDrive - Trường ĐH CNTT - University of Information Technology/Đại Học/Năm 4/Làm nhóm HK2 2022-2023/Mạng xã hội/FinalProject/RatingCSV.csv")
df.head(5)

,user_id,movie_id,rating
0,1,101,5
1,1,102,3
2,1,103,3
3,1,104,3
4,1,105,2


In [16]:
adjacency_matrix = compute_adjacency_matrix(df)
adjacency_matrix

array([[1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 0., 1., 1., 0., 1.],
       [0., 0., 1., 1., 0., 1.],
       [1., 1., 1., 0., 1., 1.],
       [1., 0., 1., 1., 1., 1.],
       [0., 0., 1., 1., 1., 0.]])

In [17]:
weight_matrix, source_target = compute_user_weight_matrix(df)

In [18]:
print(weight_matrix)

[[0. 6. 4. 3. 5. 5. 3.]
 [6. 0. 4. 3. 5. 5. 3.]
 [4. 4. 0. 3. 3. 4. 2.]
 [3. 3. 3. 0. 2. 3. 2.]
 [5. 5. 3. 2. 0. 4. 2.]
 [5. 5. 4. 3. 4. 0. 3.]
 [3. 3. 2. 2. 2. 3. 0.]]


In [19]:
print(source_target)

[[1. 2. 6.]
 [1. 3. 4.]
 [1. 4. 3.]
 [1. 5. 5.]
 [1. 6. 5.]
 [1. 7. 3.]
 [2. 3. 4.]
 [2. 4. 3.]
 [2. 5. 5.]
 [2. 6. 5.]
 [2. 7. 3.]
 [3. 4. 3.]
 [3. 5. 3.]
 [3. 6. 4.]
 [3. 7. 2.]
 [4. 5. 2.]
 [4. 6. 3.]
 [4. 7. 2.]
 [5. 6. 4.]
 [5. 7. 2.]
 [6. 7. 3.]]


In [21]:
partition = {}
the_number_of_node_in_steps(df, partition)
partition

{0: [[1], [2], [3], [4], [5], [6], [7]]}

In [22]:
louvain_algorithm(df, weight_matrix, source_target, partition, update_source_target = None)
partition

{0: [[1], [2], [3], [4], [5], [6], [7]],
 1: [[1], [2], [3], [4], [5], [6.0, 7.0]],
 2: [[1], [2], [3], [4], [5, 6.0, 7.0]],
 3: [[1], [2], [3], [4, 5, 6.0, 7.0]],
 4: [[1], [2], [3, 4, 5, 6.0, 7.0]],
 5: [[1], [2, 3, 4, 5, 6.0, 7.0]],
 6: [[1, 2, 3, 4, 5, 6.0, 7.0]]}